# CNN Architecture Comparison for CIFAR-10 Classification

## Project Overview
This project implements and compares different CNN architectures for image classification on the CIFAR-10 dataset. We'll evaluate 6 different architectural approaches to understand which design choices lead to better performance.

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np

print("TensorFlow version:", tf.__version__)
print("GPU available:", len(tf.config.list_physical_devices('GPU')) > 0)

TensorFlow version: 2.18.0
GPU available: False


## Dataset: CIFAR-10
CIFAR-10 consists of 60,000 32x32 color images in 10 classes:
- 50,000 training images
- 10,000 test images
- Classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

We normalize pixel values to [0,1] range for better training stability.

In [4]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")
print(f"Number of classes: {len(np.unique(y_train))}")

Training data shape: (50000, 32, 32, 3)
Test data shape: (10000, 32, 32, 3)
Number of classes: 10


## CNN Architecture Variations

We'll implement 6 different CNN architectures to compare their effectiveness:

### 1. **Basic CNN** (Baseline)
- Simple 3-layer CNN with max pooling
- No regularization techniques

### 2. **CNN with Dropout**
- Adds dropout layers to prevent overfitting
- Dropout rates: 0.25 after conv layers, 0.5 after dense layer

### 3. **CNN with Batch Normalization**
- Normalizes inputs to each layer
- Helps with training stability and speed

### 4. **Deeper CNN**
- More convolutional layers for increased capacity
- Tests if depth improves performance

### 5. **Different Filter Sizes**
- Varies kernel sizes (5x5, 3x3, 1x1)
- Tests impact of receptive field size

### 6. **Residual CNN**
- Simplified ResNet-like architecture
- Skip connections to help gradient flow

In [5]:
def create_basic_cnn():
    """Original basic CNN model"""
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model



## Training Methodology

### Experimental Setup:
- **Optimizer**: Adam (adaptive learning rate)
- **Loss Function**: Sparse categorical crossentropy
- **Epochs**: 5 (reduced for demonstration)
- **Batch Size**: Default (32)
- **Validation**: Using test set for validation

### Evaluation Metrics:
- **Training Accuracy**: Performance on training data
- **Validation Accuracy**: Performance on unseen test data
- **Training Loss**: How well the model fits training data
- **Final Test Accuracy**: Ultimate performance measure

Each model is trained with identical hyperparameters to ensure fair comparison.

In [6]:
def train_and_evaluate_model(model_name, model_func, epochs=5):
    print(f"\n{'='*50}")
    print(f"Training {model_name}")
    print(f"{'='*50}")

    # Create and compile model
    model = model_func()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Print model summary
    print(f"\nModel Summary for {model_name}:")
    model.summary()

    # Train model
    history = model.fit(x_train, y_train,
                       epochs=epochs,
                       validation_data=(x_test, y_test),
                       verbose=1)

    # Evaluate model
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"\nTest accuracy for {model_name}: {test_acc:.4f}")

    return history, test_acc

## Running Experiments

Now we'll train all 6 architectures and collect performance metrics. This will take approximately 10-15 minutes with GPU acceleration.



In [ ]:
# Dictionary of all models to compare
models_dict = {
    'Basic CNN': create_basic_cnn,
    'CNN with Dropout': create_cnn_with_dropout,
    'CNN with BatchNorm': create_cnn_with_batch_norm,
    'Deeper CNN': create_deeper_cnn,
    'Different Filter Sizes': create_cnn_different_filter_sizes,
    'Residual CNN': create_residual_block_cnn
}

# Train all models and collect results
results = {}
histories = {}

for model_name, model_func in models_dict.items():
    try:
        history, test_acc = train_and_evaluate_model(model_name, model_func, epochs=5)
        results[model_name] = test_acc
        histories[model_name] = history
    except Exception as e:
        print(f"Error training {model_name}: {e}")
        continue


Training Basic CNN

Model Summary for Basic CNN:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,570 (478.79 KB)

 Trainable params: 122,570 (478.79 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 32, 32,    │        896 │ input_layer_6[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_21[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 32, 32,    │      9,248 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 32, 32,    │      9,248 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_23[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ add[0][0]         │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 16, 16,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 14, 14,    │     18,496 │ max_pooling2d_12… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 7, 7, 64)  │          0 │ conv2d_24[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 5, 5, 64)  │     36,928 │ max_pooling2d_13… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_6 (Flatten) │ (None, 1600)      │          0 │ conv2d_25[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │    102,464 │ flatten_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 10)        │        650 │ dense_12[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 178,314 (696.54 KB)

 Trainable params: 178,122 (695.79 KB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 351s 222ms/step - accuracy: 0.4393 - loss: 1.5347 - val_accuracy: 0.6643 - val_loss: 0.9640
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 352s 225ms/step - accuracy: 0.6828 - loss: 0.9025 - val_accuracy: 0.6421 - val_loss: 1.0396
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 380s 224ms/step - accuracy: 0.7396 - loss: 0.7487 - val_accuracy: 0.7186 - val_loss: 0.8123
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 351s 225ms/step - accuracy: 0.7790 - loss: 0.6302 - val_accuracy: 0.7326 - val_loss: 0.7753
Epoch 5/5
1168/1563 ━━━━━━━━━━━━━━━━━━━━ 1:25 216ms/step - accuracy: 0.8121 - loss: 0.5366

## Results Analysis

The following visualizations compare all architectures across multiple metrics:

1. **Training Accuracy**: How well each model learns from training data
2. **Validation Accuracy**: How well each model generalizes to unseen data
3. **Final Test Accuracy**: Ultimate performance ranking
4. **Training Loss**: Convergence behavior



In [9]:
# Plot comparison of all models
plt.figure(figsize=(15, 10))

# (Include all the plotting code here)

plt.tight_layout()
plt.show()

<Figure size 1500x1000 with 0 Axes>

## Conclusions & Key Findings

### Performance Ranking:
[You'll fill this in after running the experiments]

### Key Insights:
1. **Best Architecture**: [Name] achieved [X]% accuracy
2. **Regularization Impact**: [Dropout/BatchNorm effects]
3. **Depth vs Performance**: [Deeper vs simpler models]
4. **Filter Size Effects**: [Different kernel sizes impact]

